In [49]:
import random
import json

In [50]:
class GrilleMotsFlechés:
    def __init__(self, lignes, colonnes, grid_init):
        self.lignes = lignes
        self.colonnes = colonnes
        self.grid = [[[None, None] for _ in range(colonnes)] for _ in range(lignes)]                                 #[Nod(a,...), Nod(a,...)] les arbres corrsepondent au moins croisant cette case
        self.grid_init = grid_init

    def draw(self):
        for i in range(self.lignes):
            for j in range(self.colonnes):
                if(self.grid[i][j][0] != None):
                    print(self.grid[i][j][0].e + '|', end='')
                elif(self.grid_init[i][j] == 1):
                    print('¤|', end = '')
                else:
                    print('0|', end = '')
            print()
        
        print()
        print()
  
    
    '''def add_def(self, definition, ligne, colonne):
        if self.grid[ligne][colonne] == '':
            self.grid[ligne][colonne] = definition
        
        else:
            self.grid[ligne][colonne].texte.append(definition.texte[0])
            self.grid[ligne][colonne].direction.append(definition.direction[0])'''

    def add_letter(self, pos, letter):
        #on met à jour les arbres de la case 
        self.grid[pos[0]][pos[1]][0] = self.grid[pos[0]][pos[1]][0].sons[letter]     
        self.grid[pos[0]][pos[1]][1] = self.grid[pos[0]][pos[1]][1].sons[letter]

        #on met à jours les arbres des cases suivantes (horizontalement et verticalement) si nécessaire

        for i in range(pos[0] + 1, self.lignes):
            if(self.grid_init[i][pos[1]] == 0):
                self.grid[i][pos[1]][1] = self.grid[pos[0]][pos[1]][1]
                break
            if(self.grid_init[i][pos[1]] == 1):
                break
        
        for j in range(pos[1] + 1, self.colonnes):
            if(self.grid_init[pos[0]][j] == 0):
                self.grid[pos[0]][j][0] = self.grid[pos[0]][pos[1]][0]
                
                break
            if(self.grid_init[pos[0]][j] == 1):
                break
            
    
    def remove_letter(self, pos):
        self.grid[pos[0]][pos[1]][0] = self.grid[pos[0]][pos[1]][0].father
        self.grid[pos[0]][pos[1]][1] = self.grid[pos[0]][pos[1]][1].father
    
    def next_pos(self, pos):
        l = pos[0]
        c = pos[1]

        if c+1 < self.colonnes:
            return (l, c+1)
        
        return (l+1, 0)

In [51]:
#e représent un character (ou bien le caractère vide également)
class Node:
    def __init__(self, e, sons, father):
        self.e = e 
        self.sons = sons
        self.father = father

    def add_node(self, e):
        if not e in self.sons:
            self.sons[e] = Node(e, {}, self)
        return self.sons[e]
    
    def add_word(self, word):
        if word == '':
            return True
        subTree = self.add_node(word[0])
        subTree.add_word(word[1:])
    
    def print(self):
        print(self.e)

        for fils in self.sons.values():
            fils.print()
            

        

def dico2tree(ordre, word_list):
    tree = Node('', {}, None)
    for word in word_list :
        if(max(ordre) > len(word)):
            print(word," : ",ordre)
        word_permute = ''.join([word[i] for i in ordre])
        tree.add_word(word_permute)
    return tree





In [52]:
class CustomErrorNoWord(Exception):
    def __init__(self, pos):
        super().__init__(pos)



def init_grid(dim_grid, grid_init, word_list_list):             #grid_init : contient les cases noires et les mots déjà remplis, world_list_list[3] contient la liste des mots de 3 lettres
    trees = {}                      #la clef ([4,2,1,3,0]) correspondra à un arbre contenant les mots de 5 lettres commencant par la 5ème lettre puis 3 puis 2 puis 4 puis 1
    mot_fleche = GrilleMotsFlechés(dim_grid[0], dim_grid[1], grid_init)
    grid = mot_fleche.grid

    #on rentre dans chaque case la taille du mot restant à partir de cette case, une pemière fois horizontalement puis verticalment 
    counter_letter = 0
    ordre_preremplie = []
    letter_preremplie = []
    coordonne_preremplie = []
    ordre_aremplir = []
    last_first_blanked_cell = [-1, -1]

    for i in range(dim_grid[0]):
        counter_letter = 0
        ordre_preremplie = []
        letter_preremplie = []
        coordonne_preremplie = []
        ordre_aremplir = []
        last_first_blanked_cell = [-1, -1]

        for j in range(dim_grid[1] + 1 ):                                                           #on parcours toujours une case de plus que l'on considère noire sur le bord
                                                                                                      
            
            if(j == dim_grid[1] or grid_init[i][j] == 1):                                           #on arrive sur une case noire, on calcule l'arbre avec le bon ordre si ce n'est pas déjà fait et on le rentre dans last_first_blanked_cell au bon étage
                if(counter_letter > 0):
                    ordre = tuple(ordre_preremplie + ordre_aremplir)
                    if(not ordre in trees):
                        trees[ordre] = dico2tree(ordre, word_list_list[counter_letter])

                    current_tree = trees[ordre]
                    for k, letter in enumerate(letter_preremplie):                                #on va descendre dans l'arbre avec les cases déjà remplie car inutile pour nous ces vilaines mécantes sorcières....
                        try:
                            current_tree = current_tree.sons[letter]
                        except: raise CustomErrorNoWord(coordonne_preremplie[k]) 
                        grid[coordonne_preremplie[k][0]][coordonne_preremplie[k][1]][0] = current_tree
                
                if(last_first_blanked_cell != [-1, -1]):
                    grid[last_first_blanked_cell[0]][last_first_blanked_cell[1]][0] = current_tree

                counter_letter = 0
                ordre_preremplie = []
                letter_preremplie = []
                coordonne_preremplie = []
                ordre_aremplir = []
                last_first_blanked_cell = [-1, -1]
                

            elif(grid_init[i][j] == 0):
                ordre_aremplir.append(counter_letter)

                if(last_first_blanked_cell == [-1, -1]):
                    last_first_blanked_cell = [i,j]

                counter_letter += 1
            
            else:
                ordre_preremplie.append(counter_letter)
                letter_preremplie.append(grid_init[i][j])
                coordonne_preremplie.append((i,j))
                
                counter_letter += 1
            


            
    
    for j in range(dim_grid[1]):
        counter_letter = 0
        ordre_preremplie = []
        letter_preremplie = []
        coordonne_preremplie = []
        ordre_aremplir = []
        last_first_blanked_cell = [-1, -1]

        for i in range(dim_grid[0] + 1):
            
            if(i == dim_grid[0] or grid_init[i][j] == 1):                                           #on arrive sur une case noire, on calcule l'arbre avec le bon ordre si ce n'est pas déjà fait et on le rentre dans last_first_blanked_cell au bon étage
                if(counter_letter > 0):
                    ordre = tuple(ordre_preremplie + ordre_aremplir)
                    if(not ordre in trees):
                        trees[ordre] = dico2tree(ordre, word_list_list[counter_letter])

                    current_tree = trees[ordre]
                    for k, letter in enumerate(letter_preremplie):                                #on va descendre dans l'arbre avec les cases déjà remplie car inutile pour nous ces vilaines mécantes sorcières....
                        try:
                            current_tree = current_tree.sons[letter]
                        except: raise CustomErrorNoWord(coordonne_preremplie[k])
                        grid[coordonne_preremplie[k][0]][coordonne_preremplie[k][1]][1] = current_tree
                
                if(last_first_blanked_cell != [-1, -1]):
                    grid[last_first_blanked_cell[0]][last_first_blanked_cell[1]][1] = current_tree

                counter_letter = 0
                ordre_preremplie = []
                letter_preremplie = []
                coordonne_preremplie = []
                ordre_aremplir = []
                last_first_blanked_cell = [-1, -1]
                

            elif(grid_init[i][j] == 0):
                ordre_aremplir.append(counter_letter)

                if(last_first_blanked_cell == [-1, -1]):
                    last_first_blanked_cell = [i,j]

                counter_letter += 1            
            else:
                ordre_preremplie.append(counter_letter)
                letter_preremplie.append(grid_init[i][j])
                coordonne_preremplie.append((i,j))
        
                counter_letter += 1


            

    return mot_fleche

In [53]:
class CustomError(Exception):
    def __init__(self, grid):
        super().__init__(grid)


def complete_grid(mot_fleche, pos):
    if pos == (mot_fleche.lignes, 0):
        mot_fleche.draw()
        raise CustomError(mot_fleche.grid)
    
    next_pos = mot_fleche.next_pos(pos)

    
    tree_h = mot_fleche.grid[pos[0]][pos[1]][0]
    tree_v = mot_fleche.grid[pos[0]][pos[1]][1]

 
    if mot_fleche.grid_init[pos[0]][pos[1]] != 0:                                   #il s'agit d'une case contenant une défintion ou un mot prérempli, on ne la touche pas
        complete_grid(mot_fleche, next_pos)
    
    else:

        #print(pos)
        #print(tree_h, " : ", tree_h.sons)
        #print(tree_v, " : ", tree_v.sons)
        #mot_fleche.draw()
        
        # On va récupérer l'intesection des deux listes de lettres possibles afin de tester toutes les possibilités qui nous sont offertes 
        
        poss_v = set(tree_v.sons.keys())
        poss_h = set(tree_h.sons.keys())

        poss = poss_v & poss_h
        poss_list = list(poss)
        random.shuffle(poss_list)

        for scenario in poss_list:
            mot_fleche.add_letter(pos, scenario)
            complete_grid(mot_fleche, next_pos)
            mot_fleche.remove_letter(pos)



In [54]:
def generate_word_list_list(adresse_fichier, editor_words):
    with open(adresse_fichier, 'r') as fichier:
        lignes = fichier.readlines()

    word_list = [ligne.strip() for ligne in lignes]

    #word_list = list(dictionary['solutions'].values())                    #Bypass pour utiliser le dictionnaire d'Achille

    word_list += editor_words

    length_list = [len(word) for word in word_list]

    max_length = max(length_list)

    word_list_list = [[] for _ in range(max_length+1)]

    for i in range(len(word_list)):
        word_list_list[len(word_list[i])].append(word_list[i].upper())

    return word_list_list

In [55]:
def gridMoche2Jolie(grid):
    lignes = len(grid)
    colonnes = len(grid[0])

    grid_jolie = [[[0] for _ in range(colonnes)] for _ in range(lignes)]

    for i, ligne in enumerate(grid):
        for j, case in enumerate(ligne):
            if(grid[i][j][0] == None):
                grid_jolie[i][j] = 1
            else:
                grid_jolie[i][j] = grid[i][j][0].e
    
    return grid_jolie

def generate(dim_grid, grid_init, adresse_fichier, editor_words):
    word_list_list = generate_word_list_list(adresse_fichier, editor_words)
    mot_fleche = init_grid(dim_grid, grid_init, word_list_list)
    print("ca commence à calculer :")
    try:
        complete_grid(mot_fleche, (0,0))
    except CustomError as grid:
        grille = grid.args[0]
        print(grille)
        grille = gridMoche2Jolie(grille)
        return grille

    

In [56]:
def words_in_grid(grid):
    word_list_h = []
    word_list_v = []

    current_word = ''
    for i in range(len(grid)):
        for j in range(len(grid[0])):
            if(grid[i][j] == 1 or grid[i][j] == 0):
                if(len(current_word) > 1):
                    word_list_h.append(current_word)
                current_word = ''
            else:
                current_word += grid[i][j]
        if(len(current_word) > 1):
            word_list_h.append(current_word)
        current_word = ''
    
    current_word = ''
    for j in range(len(grid[0])):
        for i in range(len(grid)):
            if(grid[i][j] == 1 or grid[i][j] == 0):
                if(len(current_word) > 1):
                    word_list_v.append(current_word)
                current_word = ''
            else:
                current_word += grid[i][j]
        if(len(current_word) > 1):
            word_list_v.append(current_word)
        current_word = ''

    return word_list_h, word_list_v

In [57]:
def words2definitions(words, dictionary):
    definitions = []
    for word in words:
        word_lower = word.lower()
        print(word_lower)
        if(word_lower in dictionary['solutions']):
            #word_id = dictionary['solutions'][word_lower]                      petit problème d'ID pour les solutions, gloabalement c'est le mot qui convient
            def_id = random.choice(dictionary['solutions_id_to_definitions_id'][word_lower])
            #definitions.append(dictionary['definitions'][def_id])              petit problème d'ID pour les défs, il y a des ID correspondant à aucune déf
            definitions.append(def_id)                          
        else:
            definitions.append("Pas de déf pour ce mot là")
    return definitions

In [58]:
dim_grid = 10,10
grid_init = [[0 for _ in range(dim_grid[1])] for _ in range(dim_grid[0])]

with open('mots_fleches.json', 'r') as file:
    dictionary = json.load(file)

print(dictionary['definitions'].keys())

grid_init[0][0] = 1
grid_init[0][2] = 1
grid_init[0][4] = 1
grid_init[0][6] = 1
grid_init[0][8] = 1

grid_init[2][0] = 1
grid_init[2][6] = 1

grid_init[3][5] = 1

grid_init[4][0] = 1
grid_init[4][7] = 1

grid_init[5][8] = 1

grid_init[6][0] = 1
grid_init[6][4] = 1
grid_init[6][6] = 1

grid_init[7][2] = 1
grid_init[7][3] = 1

grid_init[8][0] = 1

grid_init[9][5] = 1

adresse_fichier = 'fr.txt'

dict_keys(['ceinture-a-tokyo', 'ceinture-verte', 'elle-fait-le-tour-du-jardin', 'figurine-de-terre-cuite', 'garnie-dune-bordure', 'lisiere-de-bois', 'bois-des-bois', 'elle-fait-le-tour-du-bois', 'abime-sur-les-bords', 'salades-vertes', 'entame-le-capital', 'alteration-des-boissons', 'ancienne-capitale-des-gaules', 'narine-de-baleine', 'grand-canal-devacuation', 'bois-des-indes', 'donne-une-meilleure-evacuation', 'les-doigts-dans-le-nez', 'il-sonne-lhallali', 'il-circule-a-tokyo', 'passage-de-canal', 'garnier-ou-bastille', 'genies-des-bois', 'grand-cerf-des-pays-du-nord', 'tailler-un-diamant', 'fait-ceinture', 'il-circule-au-cambodge', 'arrive-au-bout-des-doigts', 'serrer-la-ceinture', 'vetements-amples', 'balance-sur-les-flots-de-malaisie', 'mammifere-aquatique', 'se-fait-en-tournant', 'a-la-portee-de-tous', 'animal-aquatique', 'serrer-contre-soi', 'accent-verbal', 'oeuvre-nippone', 'garnies-de-metal', 'aise', 'alterations-musicales', 'aux-environs', 'un-adverbe-qui-en-redemande', 'tel

In [59]:
import tkinter as tk
from tkinter import messagebox

class CrosswordUserInterface:
    def __init__(self, grille_affichage, grille_modifiable):
        self.grille_affichage = grille_affichage
        self.grille_modifiable = grille_modifiable
        self.dimension = (len(grille_modifiable),len(grille_modifiable[0]))

        self.editor_words = []

        self.words_in_grid_v = []
        self.words_in_grid_h = []

        self.def_h = []
        self.def_v = []

        
        # Créer la fenêtre principale
        self.root = tk.Tk()
        self.root.title("Mot Croisé")

        # Dictionnaires pour stocker les widgets
        self.entries_modifiable = {}
        self.labels_display = {}

        # Créer des Frames pour les grilles
        self.frame_modifiable = tk.Frame(self.root)
        self.frame_modifiable.grid(row=1, column=0, padx=15, pady=15)

        self.frame_affichage = tk.Frame(self.root)
        self.frame_affichage.grid(row=1, column=1, padx=10, pady=10)

        #Créer un Frame pour afficher les defs :
        self.frame_def = tk.Frame(self.root)
        self.frame_def.grid(row = 1, column=2, padx=10, pady=10)

        self.message_def = tk.Message(self.frame_def, text= "Définitions :   \n  -Horizontale :  \n  -Verticale :",
                     width=1000, font=("Arial", 11))
        
        self.message_def.pack()

        # Zone pour entrer le nombre de lignes et de colonnes
        tk.Label(self.root, text="Nombre de lignes:").grid(row=0, column=0, padx=10, pady=10, sticky="e")
        self.entry_lines = tk.Entry(self.root)
        self.entry_lines.grid(row=0, column=1, padx=10, pady=10)

        tk.Label(self.root, text="Nombre de colonnes:").grid(row=0, column=2, padx=10, pady=10, sticky="e")
        self.entry_columns = tk.Entry(self.root)
        self.entry_columns.grid(row=0, column=3, padx=10, pady=10)

        # Bouton pour appliquer les dimensions
        self.bouton_appliquer = tk.Button(self.root, text="Appliquer Dimensions", command=self.appliquer_dimensions)
        self.bouton_appliquer.grid(row=0, column=4, padx=10, pady=10)

        # Bouton pour mettre à jour la grille modifiable
        self.bouton_mise_a_jour = tk.Button(self.root, text="Mettre à jour la grille", command=self.mettre_a_jour_grille)
        self.bouton_mise_a_jour.grid(row=2, columnspan=5, pady=10)
    
    # Fonction pour forcer les majuscules lors de la saisie
    def forcer_majuscules(self,event):
        if event.widget.cget('bg') != '#404040':  # Ne pas forcer les majuscules si la case est noire
            event.widget.delete(0, tk.END)
            event.widget.insert(0, event.char.upper())

    # Fonction pour gérer le clic droit et transformer une case en noire ou en vide
    def transformer_case(self, event, i, j):
        entry = self.entries_modifiable[(i, j)]
        if entry.cget('bg') == '#404040':  # Si déjà noire foncée, la rendre vide
            entry.config(bg="white", fg="black", state="normal")
            entry.delete(0, tk.END)  # Supprimer le texte s'il y en a
            self.grille_modifiable[i][j] = 0
        else:  # Si pas noire, la rendre noire foncée
            entry.config(bg="#404040", fg="white", state="normal")
            entry.delete(0, tk.END)  # Supprimer le texte pour ne garder que le noir
            self.grille_modifiable[i][j] = 1

    # Fonction pour mettre à jour les grilles en fonction des dimensions fournies
    def appliquer_dimensions(self):
        try:
            lignes = int(self.entry_lines.get())
            colonnes = int(self.entry_columns.get())
        except ValueError:
            messagebox.showerror("Erreur", "Les valeurs de lignes et colonnes doivent être des entiers.")
            return

        # Mise à jour des grilles
        self.grille_modifiable = [[0 for _ in range(colonnes)] for _ in range(lignes)]
        self.grille_affichage = [[0 for _ in range(colonnes)] for _ in range(lignes)]

        self.dimension = (lignes, colonnes)

        self.entries_modifiable = {}
        self.labels_display = {}
        
        # Réinitialiser les Frames
        for widget in self.frame_modifiable.winfo_children():
            widget.destroy()
        for widget in self.frame_affichage.winfo_children():
            widget.destroy()

        # Recréer les grilles avec les nouvelles dimensions
        self.creer_grille_modifiable()
        self.creer_grille_affichage()


    # Fonction pour créer la grille d'affichage
    def creer_grille_affichage(self):
        print(self.grille_affichage)
        for i, ligne in enumerate(self.grille_affichage):
            for j, lettre in enumerate(ligne):
                label = tk.Label(self.frame_affichage, font=("Helvetica", 24), width=3, justify='center', borderwidth=2, relief="solid")
                label.grid(row=i, column=j, padx=5, pady=5)

                # Si la case est noire (définie comme 1), alors la colorer en noir foncé
                if lettre == 1:
                    label.config(bg="#404040", fg="white")
                elif lettre == 0:  # Comparer avec 0 pour laisser la case vide
                    label.config(bg="white", fg="black")
                else:
                    label.config(text=lettre.upper(), bg="white", fg="black")  # Assurer que les lettres sont en majuscule

                # Sauvegarder le label dans le dictionnaire avec sa position comme clé
                self.labels_display[(i, j)] = label

    # Fonction pour mettre à jour la grille d'affichage
    def mettre_a_jour_affichage(self):
        for (i, j), label in self.labels_display.items():
            valeur = self.grille_affichage[i][j]
            if valeur == 1:
                label.config(bg="#404040", fg="white", text="")
            elif valeur == 0:
                label.config(bg="white", fg="black", text="")
            else:
                label.config(bg="white", fg="black", text=valeur.upper())

    # Fonction pour mettre à jour la grille modifiable avec les nouvelles lettres
    def mettre_a_jour_grille(self):
        for (i, j), entry in self.entries_modifiable.items():
            nouvelle_lettre = entry.get().upper()
            if entry.cget('bg') == '#404040':  # Case noire foncée
                self.grille_modifiable[i][j] = 1
            else:
                self.grille_modifiable[i][j] = nouvelle_lettre if nouvelle_lettre != '' else 0
        print("Nouvelle grille modifiable mise à jour :", self.grille_modifiable)

        for widget in self.frame_affichage.winfo_children():
            widget.destroy()

        editor_words_h, editor_words_v = words_in_grid(self.grille_modifiable)
        self.editor_words = editor_words_h + editor_words_v
        
        self.creer_grille_modifiable()              #Permet de faire table rase des cases qui ont été mise en rouge précédement (oui c'est vachement violent mais franchement ça prend pas bcp de temps en vrai)

        try:
            self.grille_affichage = generate(self.dimension, self.grille_modifiable, adresse_fichier, self.editor_words)
        except CustomErrorNoWord as pos:
            print("Aucune grille n'est possible avec vos lettres")                                                                                                    #si une lettre empêche tout mot de se former, on la met en rouge pour aider à l'édition
            i,j = pos.args[0]
            entry = self.entries_modifiable[(i,j)]
            entry.config(bg="red")
            return
        
        self.words_in_grid_h, self.words_in_grid_v = words_in_grid(self.grille_affichage)

        
        self.def_h = words2definitions(self.words_in_grid_h, dictionary)
        self.def_v = words2definitions(self.words_in_grid_v, dictionary)

        message = "Définitions :   \n  -Horizontale :" 

        
        for i in range(len(self.words_in_grid_h)):
            message += (" \n    " + self.words_in_grid_h[i] + " : " + self.def_h[i])
        message += "\n  -Verticale :"
        for i in range(len(self.words_in_grid_v)):
            message += (" \n    " + self.words_in_grid_v[i] + " : " + self.def_v[i])
        
        self.message_def.config(text = message)
        

        print(self.words_in_grid_h)
        print(self.words_in_grid_v)

        self.creer_grille_affichage()
        self.mettre_a_jour_affichage()
    
    # Fonction pour créer la grille modifiable
    def creer_grille_modifiable(self):
        for i, ligne in enumerate(self.grille_modifiable):
            for j, lettre in enumerate(ligne):
                entry = tk.Entry(self.frame_modifiable, font=("Helvetica", 24), width=3, justify='center', borderwidth=2, relief="solid")
                entry.grid(row=i, column=j, padx=5, pady=5)
                
                # Si la case est noire (définie comme 1), alors la colorer en noir foncé
                if lettre == 1:
                    entry.config(bg="#404040", fg="white", state="normal")  # Normal pour permettre le clic droit
                elif lettre != 0:  # Comparer avec 0 pour remplir la case si elle n'est pas vide
                    entry.insert(0, lettre)
                    entry.config(fg="black")
                
                # Lier l'événement KeyRelease à la fonction forcer_majuscules pour forcer la saisie en majuscule, sauf si la case est noire
                entry.bind("<KeyRelease>", self.forcer_majuscules)
                
                # Lier l'événement clic droit pour transformer la case en noire ou en vide
                entry.bind("<Button-3>", lambda event, i=i, j=j: self.transformer_case(event, i, j))
                
                # Sauvegarder l'entry dans le dictionnaire avec sa position comme clé
                self.entries_modifiable[(i, j)] = entry

    #Fonction pour démarrer l'interface graphique
    def start(self):
        # Créer les grilles initiales avec les dimensions par défaut
        self.creer_grille_modifiable()
        self.creer_grille_affichage()

        self.root.mainloop()

In [60]:
grille_modifiable_test = grid_init
grille_affichage_test  = [[0 for _ in range(dim_grid[1])] for _ in range(dim_grid[0])]

cross_word = CrosswordUserInterface(grille_affichage_test, grille_modifiable_test)
cross_word.start()

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
Nouvelle grille modifiable mise à jour : [[1, 0, 1, 0, 1, 0, 1, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 'A', 0, 0, 0, 1, 0, 0, 0], [0, 0, 'A', 0, 0, 1, 0, 0, 0, 0], [1, 0, 'A', 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [1, 0, 0, 0, 1, 0, 1, 0, 0, 0], [0, 0, 1, 1, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]]
Aucune grille n'est possible avec vos lettres
